In [1]:
import time
import datetime
import re
from itertools import product
from collections import OrderedDict
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pymongo
from pymongo import MongoClient
from pprint import pprint

In [2]:
def writeLog(log_text, function_name):
    with open('web_parcer.log', 'a') as file:
        file.write(str(datetime.datetime.now()) + ":\t" + function_name + ': \t' + log_text + '\n')


def goToPage(query, driver):
    start_loading = datetime.datetime.now()
    try:
        driver.set_page_load_timeout(120)
        driver.get("https://www.aviasales.ru/search/{}".format(query))
    except WebDriverException:
        writeLog('Page loading timeout :' + query, 'goToPage')
    
    try:
        time.sleep(3)    
        wait_seconds = 600
        wait = WebDriverWait(driver, wait_seconds)
        wait.until(EC.invisibility_of_element((By.CLASS_NAME, "countdown")))
        loading_duration = (datetime.datetime.now() - start_loading).seconds
        printDelimiter('*')
        print("$$$$$   page loaded for {} seconds   $$$$$".format(loading_duration))
        time.sleep(1)
    except:
        print("\x1b[31m\"CAN'T WAIT THE PAGE LOAD ({} seconds)\"\x1b[0m".format(wait_seconds))
    

def closeExtraTab(driver): # В предположении что драйвер находится на рекламном окне после его открытия
    num_of_tabs = len(driver.window_handles)
    if num_of_tabs == 2:
        main_window_handle = None
        if driver.current_window_handle == driver.window_handles[0]:
            driver.switch_to_window(driver.window_handles[1])
            main_window_handle = driver.window_handles[0]
        elif driver.current_window_handle == driver.window_handles[1]:
            driver.switch_to_window(driver.window_handles[0])
            main_window_handle = driver.window_handles[1]
        title_of_closed_tab = driver.title
        driver.close()
        driver.switch_to_window(main_window_handle)
        writeLog("Extra tab *{}* has been closed".format(title_of_closed_tab), 'closeExtraTab')
    
    
def printDelimiter(char):
    delimiter = ''
    for i in range(118):
        delimiter += char
    print(delimiter)
    
    
def getCollection():
    client = MongoClient()
    db = client.aviasales_database # getting a database
    return db.ticket_collection_v2

In [3]:
#hardcoded_year = None################

def processPage(driver, collection):
    tickets = driver.find_elements_by_class_name("ticket")
    tickets = removeAdvertisingTickets(tickets)
    tickets = removeAbsentTickets(tickets)
    print(driver.title[-9:], '\n')
    for ticket in tickets:
        collection.insert_one(processTicket(ticket))
        closeExtraTab(driver)
        
        
def removeAdvertisingTickets(tickets):
    for ticket in tickets:
        ticket_opener_text = ticket.find_element_by_class_name("ticket__expand-button").text
        if ticket_opener_text == 'Реклама':
            tickets.remove(ticket)
    return tickets


def removeAbsentTickets(tickets):
    for ticket in tickets:
        ticket_price = (ticket.find_element_by_class_name("buy-button__price").text).replace('\u2009', '')
        if ticket_price == '':
            tickets.remove(ticket)
    return tickets
        
        
def processTicket(ticket):
    ticket_price = int((ticket.find_element_by_class_name("buy-button__price-num").text).replace('\u2009', ''))
    
    ticket_opener = ticket.find_element_by_class_name("ticket__expand-button")
    ticket_opener.click()
    
    ticket_content = ticket.find_element_by_class_name("ticket__content")
    
    flights = ticket_content.find_elements_by_class_name("ticket-segment")
    
    print("crawl_date_time: ", datetime.datetime.now())                    # debug
    print("price: ", ticket_price)                                         # debug
    
    post = dict()
    post['crawl_date_time'] = datetime.datetime.now()
    post['price'] = ticket_price
    post['flights'] = list()
    
    for flight in flights:
        post['flights'].append(processFlight(post, flight))
    
    return post

        
def processFlight(post, flight):
    flight_info = dict()
    flight_info['details'] = list()
    
    general_flight_info = getGeneralFlightInfo(flight)
    flight_info['city'] = general_flight_info[0]
    flight_info['date_time'] = general_flight_info[1]
    flight_info['total_time'] = general_flight_info[2]
    
    #nonlocal hardcoded_year
    #hardcoded_year = flight_general_info['date_time']['from'].year#############################
    
    
    detailed_flight_info = getDetailedFlightInfo(flight)
    for info in detailed_flight_info:
        flight_info['details'].append(info)
        
    flight_info_order = ['total_time', 'date_time', 'city']        # debug
    for info in flight_info_order:                                 # debug
        print('\t' + info + ": ", flight_info[info])               # debug
        
    print('\n')

    return flight_info
    
    
def getGeneralFlightInfo(flight):
# Общая информация
    flight_from = getGeneralPointInfo(flight, "segment-route__endpoint.origin")

    flight_info_path = getGeneralPathInfo(flight)
    
    flight_to = getGeneralPointInfo(flight, "segment-route__endpoint.destination")
    
    flight_info_city = {'from' : flight_from[0],
                        'to' : flight_to[0]}
    flight_info_date_time = {'from' : flight_from[1],
                             'to' : flight_to[1]}
    
    return flight_info_city, flight_info_date_time, flight_info_path


def getGeneralPathInfo(flight):
    flight_path = flight.find_element_by_class_name("segment-route__route_wrap")
    flight_total_time = re.split(' ', flight_path.find_element_by_class_name("segment-route__duration").text)
    flight_total_time_info = getHoursAndMinutes(flight_total_time)
    return flight_total_time_info
    
    
def getHoursAndMinutes(time_string):
    hours = 0
    minutes = 0
    days = 0
    for word in time_string:
        if word[-1] == 'ч':
            hours = word[:-1]
        elif word[-1] == 'м':
            minutes = word[:-1]
        elif word[-1] == 'д':
            days = word[:-1]
    return {'days' : days, 'hours' : hours, 'minutes' : minutes}
    
    
def getGeneralPointInfo(flight, segment_route):
    flight_point = flight.find_element_by_class_name(segment_route)
    flight_city = flight_point.find_element_by_class_name("segment-route__city").text
    time_point = re.split(':', flight_point.find_element_by_class_name("segment-route__time").text)
    date_point = re.split(' ', flight_point.find_element_by_class_name("segment-route__date").text)
    flight_date_time_point = datetime.datetime(int(date_point[2][:-1]), 
                                               translateDate(date_point[1]), 
                                               int(date_point[0]), 
                                               int(time_point[0]), 
                                               int(time_point[1]))
    return flight_city, flight_date_time_point

    
def translateDate(month):
    date_translator = {'янв' : 1, 'фев' : 2, 'мар' : 3, 'апр' : 4 ,'май' : 5 ,'июн' : 6, 'июл' : 7, 'авг' : 8,
                       'сен' : 9, 'окт' : 10, 'ноя' : 11, 'дек' : 12}
    return date_translator[month]
    
    
def getDetailedFlightInfo(flight):
    flight_info = list()
    try:
    # Детали перелета
        time.sleep(1)
    
        flight_itinerary = flight.find_element_by_class_name("ticket-segment__details")

        # Flight legs
        flight_legs = flight_itinerary.find_elements_by_class_name("ticket-segment__flight")
        flight_legs_info = list()

        for flight_leg in flight_legs:
            flight_legs_info.append(getDetailedFlightLegInfo(flight_leg))


        # Flight stops    
        flight_stops = flight_itinerary.find_elements_by_class_name("ticket-segment__stop")
        flight_stops_info = list()

        for flight_stop in flight_stops:
            flight_stops_info.append(getDetailedFlightStopInfo(flight_stop))


        # Миксую через один инфу по остановкам и инфу по пересадкам (для восстановления хронологии в выводе)
        flight_info = mixFlightInfo(flight_legs_info, flight_stops_info)

        #for info in flight_info:                                               # debug
        #    if info['type'] == 'leg':                                          # debug
        #        print("\n\tflight date time: ", info['date_time'],             # debug
        #              "\n\t\tairport: ", info['airport'],                      # debug
        #              "\n\t\tairline: ", info['airline'],                      # debug
        #              "\n\t\tflight number: ", info['flight_number'])          # debug
        #    elif info['type'] == 'stop':                                       # debug
        #        print("\n\tstop time: ", info['time'],                         # debug
        #              "\n\tstop place: ", info['place'])                       # debug

        #print("\n")                                                        # debug
    except NoSuchElementException as exception:
        #print("\x1b[31m\"red\"\x1b[0m")
        print("\x1b[31m\"Element not found (getDetailedFlightInfo): ", exception, "\"\x1b[0m")
    return flight_info 
    
                                 
def mixFlightInfo(flight_legs_info, flight_stops_info):
    flight_info = [flight_stops_info[i//2] 
                   if i%2 else flight_legs_info[i//2] 
                   for i in range(len(flight_stops_info)*2+1)]
    return flight_info
                                 
    
def getDetailedFlightStopInfo(flight_stop):
    flight_attrs_tmp = dict()
    flight_attrs_tmp['place'] = flight_stop.find_element_by_class_name("ticket-stop__place").text
    flight_stop_time = re.split(' ', flight_stop.find_element_by_class_name("ticket-stop__time").text)
    flight_attrs_tmp['time'] = getHoursAndMinutes(flight_stop_time)
    flight_attrs_tmp['type'] = 'stop'
    return flight_attrs_tmp
    

def getDetailedFlightLegInfo(flight_leg):
    try:
        flight_attrs_tmp = dict()
        
        flight_attrs_tmp['type'] = 'leg'

        flight_from_info = flight_leg.find_elements_by_class_name("ticket-flight__route-info")[0]
        flight_to_info = flight_leg.find_elements_by_class_name("ticket-flight__route-info")[1]

        flight_date_time_from = getDetailedFlightLegDatetimeInfo(flight_from_info)
        flight_date_time_to = getDetailedFlightLegDatetimeInfo(flight_to_info)
        
        flight_attrs_tmp['date_time'] = {'from' : flight_date_time_from,
                                         'to' : flight_date_time_to}

        flight_attrs_tmp['airport'] = getDetailedFlightLegAirportInfo(flight_from_info, flight_to_info)

        flight_airline = flight_leg.find_element_by_class_name("ticket-carrier")
        flight_attrs_tmp['airline'] = flight_airline.find_element_by_css_selector('img').get_attribute('alt')
        
        flight_number = flight_leg.find_element_by_class_name("ticket-flight__title")
        flight_attrs_tmp['flight_number'] = flight_number.text[5:]
    
    except NoSuchElementException as exception:
        print("\x1b[31m\"Element not found (getDetailedFlightLegInfo): ", exception, "\"\x1b[0m")
    return flight_attrs_tmp


def getDetailedFlightLegAirportInfo(flight_from_info, flight_to_info):
    flight_airport_name_from = flight_from_info.find_element_by_class_name("ticket-flight__name").text
    flight_airport_iata_from = flight_from_info.find_element_by_class_name("ticket-flight__iata").text
    flight_airport_name_to = flight_to_info.find_element_by_class_name("ticket-flight__name").text
    flight_airport_iata_to = flight_to_info.find_element_by_class_name("ticket-flight__iata").text
    flight_airport_info = {'from' : 
                                   {'name' : flight_airport_name_from,
                                    'iata' : flight_airport_iata_from},
                           'to' : 
                                   {'name' : flight_airport_name_to,
                                    'iata' : flight_airport_iata_to}
                          }
    return flight_airport_info


def getDetailedFlightLegDatetimeInfo(flight_info):
    
    flight_time = re.split(':', flight_info.find_element_by_class_name("ticket-flight__departure-time").text)
    flight_date = re.split(' ', flight_info.find_element_by_class_name("ticket-flight__departure-date").text)
    
    #ATTENTION: создаю багу того что год в дате проставляется как год даты начала поездки
    flight_date_time= datetime.datetime(2019, #######hardcode######################################
                                        translateDate(flight_date[1][:-1]),
                                        int(flight_date[0]),
                                        int(flight_time[0]),
                                        int(flight_time[1])) 
    return flight_date_time


In [22]:
driver = webdriver.Firefox()
driver.get('https://www.aviasales.ru/search/MOW1307STO1')

In [18]:
def crawlByList(dates_destinations_pairs, driver): 
    collection = getCollection()
    query = 'MOW'
    while len(dates_destinations_pairs) > 0:
        pair = dates_destinations_pairs[0]
        query = 'MOW' + pair[0] + pair[1] + '1'
        goToPage(query, driver)
        processPage(driver, collection)
        dates_destinations_pairs.remove(pair)

#При остановке паука можно продолжить поиск с той пары на которой все сломалось
def makeDateDestinationPairs(dates, destinations): # dates - что то в духе ['1304','1404', ...]
    return list(product(dates, destinations))

#При перезапуске кернела, можно обрезать начало массива пар, которое уже пройдено краулером
def skipPairsBefore(pair_array, last_seen_pair):
    idx = pair_array.index(last_seen_pair)
    return pair_array[idx:]

In [25]:
# НЕ ЗАПУСКАТЬ ПРИ ПОЛОМКЕ (создаст заново все пары городов и дат)
city_time_pairs = makeDateDestinationPairs(time_list_september[:15], city_list) 

In [26]:
print(city_time_pairs[:50])

[('0109', 'ALC'), ('0109', 'AMS'), ('0109', 'ANR'), ('0109', 'BCN'), ('0109', 'BFS'), ('0109', 'BER'), ('0109', 'BRN'), ('0109', 'BLQ'), ('0109', 'BRE'), ('0109', 'BRQ'), ('0109', 'OST'), ('0109', 'BRU'), ('0109', 'BUD'), ('0109', 'VLC'), ('0109', 'VIE'), ('0109', 'VCE'), ('0109', 'VRN'), ('0109', 'HAM'), ('0109', 'HAJ'), ('0109', 'GLA'), ('0109', 'GOT'), ('0109', 'DUB'), ('0109', 'DUS'), ('0109', 'GVA'), ('0109', 'SZG'), ('0109', 'IBZ'), ('0109', 'INN'), ('0109', 'KLV'), ('0109', 'CBG'), ('0109', 'ORC'), ('0109', 'CGN'), ('0109', 'LCA'), ('0109', 'LEJ'), ('0109', 'LYS'), ('0109', 'LON'), ('0109', 'MAD'), ('0109', 'PMI'), ('0109', 'AGP'), ('0109', 'MAN'), ('0109', 'MIL'), ('0109', 'MPL'), ('0109', 'MUC'), ('0109', 'NAP'), ('0109', 'ECN'), ('0109', 'NCE'), ('0109', 'NUE'), ('0109', 'PMO'), ('0109', 'PAR'), ('0109', 'PFO'), ('0109', 'PEG')]


In [27]:
# ЗАПУСКАТЬ ТОЛЬКО при перезапуске кернела
city_time_pairs = skipPairsBefore(city_time_pairs, ('0609', 'NAP'))
print(city_time_pairs[:50])

[('0609', 'NAP'), ('0609', 'ECN'), ('0609', 'NCE'), ('0609', 'NUE'), ('0609', 'PMO'), ('0609', 'PAR'), ('0609', 'PFO'), ('0609', 'PEG'), ('0609', 'PRG'), ('0609', 'ROM'), ('0609', 'RTM'), ('0609', 'SVQ'), ('0609', 'STO'), ('0609', 'SXB'), ('0609', 'QCN'), ('0609', 'TRN'), ('0609', 'FLR'), ('0609', 'ZRH'), ('0609', 'EDI'), ('0609', 'EIN'), ('0709', 'ALC'), ('0709', 'AMS'), ('0709', 'ANR'), ('0709', 'BCN'), ('0709', 'BFS'), ('0709', 'BER'), ('0709', 'BRN'), ('0709', 'BLQ'), ('0709', 'BRE'), ('0709', 'BRQ'), ('0709', 'OST'), ('0709', 'BRU'), ('0709', 'BUD'), ('0709', 'VLC'), ('0709', 'VIE'), ('0709', 'VCE'), ('0709', 'VRN'), ('0709', 'HAM'), ('0709', 'HAJ'), ('0709', 'GLA'), ('0709', 'GOT'), ('0709', 'DUB'), ('0709', 'DUS'), ('0709', 'GVA'), ('0709', 'SZG'), ('0709', 'IBZ'), ('0709', 'INN'), ('0709', 'KLV'), ('0709', 'CBG'), ('0709', 'ORC')]


In [28]:
crawlByList(city_time_pairs, driver) # main function

**********************************************************************************************************************
$$$$$   page loaded for 123 seconds   $$$$$
MOW → NAP 

crawl_date_time:  2019-04-22 12:35:40.389590
price:  8788
	total_time:  {'minutes': '40', 'days': 0, 'hours': '6'}
	date_time:  {'to': datetime.datetime(2019, 9, 6, 10, 40), 'from': datetime.datetime(2019, 9, 6, 5, 0)}
	city:  {'to': 'Неаполь', 'from': 'Москва'}


crawl_date_time:  2019-04-22 12:35:43.409089
price:  8788
	total_time:  {'minutes': '55', 'days': 0, 'hours': '6'}
	date_time:  {'to': datetime.datetime(2019, 9, 6, 22, 35), 'from': datetime.datetime(2019, 9, 6, 16, 40)}
	city:  {'to': 'Неаполь', 'from': 'Москва'}


crawl_date_time:  2019-04-22 12:35:46.298647
price:  8788
	total_time:  {'minutes': '50', 'days': 0, 'hours': '10'}
	date_time:  {'to': datetime.datetime(2019, 9, 6, 14, 50), 'from': datetime.datetime(2019, 9, 6, 5, 0)}
	city:  {'to': 'Неаполь', 'from': 'Москва'}


crawl_date_time:  2019-04-2

crawl_date_time:  2019-04-22 12:38:22.795238
price:  10864
	total_time:  {'minutes': '15', 'days': 0, 'hours': '23'}
	date_time:  {'to': datetime.datetime(2019, 9, 7, 8, 40), 'from': datetime.datetime(2019, 9, 6, 10, 25)}
	city:  {'to': 'Ницца', 'from': 'Москва'}


**********************************************************************************************************************
$$$$$   page loaded for 30 seconds   $$$$$
MOW → NUE 

crawl_date_time:  2019-04-22 12:38:59.203216
price:  8709
	total_time:  {'minutes': '30', 'days': 0, 'hours': '5'}
	date_time:  {'to': datetime.datetime(2019, 9, 6, 20, 15), 'from': datetime.datetime(2019, 9, 6, 15, 45)}
	city:  {'to': 'Нюрнберг', 'from': 'Москва'}


crawl_date_time:  2019-04-22 12:39:02.880013
price:  9243
	total_time:  {'minutes': '10', 'days': 0, 'hours': '14'}
	date_time:  {'to': datetime.datetime(2019, 9, 7, 9, 50), 'from': datetime.datetime(2019, 9, 6, 20, 40)}
	city:  {'to': 'Нюрнберг', 'from': 'Москва'}


crawl_date_time:  2019-0

crawl_date_time:  2019-04-22 12:43:17.804032
price:  6221
	total_time:  {'minutes': '25', 'days': 0, 'hours': '10'}
	date_time:  {'to': datetime.datetime(2019, 9, 6, 19, 10), 'from': datetime.datetime(2019, 9, 6, 9, 45)}
	city:  {'to': 'Прага', 'from': 'Москва'}


crawl_date_time:  2019-04-22 12:43:20.588707
price:  6221
	total_time:  {'minutes': '20', 'days': 0, 'hours': '15'}
	date_time:  {'to': datetime.datetime(2019, 9, 6, 19, 10), 'from': datetime.datetime(2019, 9, 6, 4, 50)}
	city:  {'to': 'Прага', 'from': 'Москва'}


crawl_date_time:  2019-04-22 12:43:23.601841
price:  6221
	total_time:  {'minutes': '35', 'days': 0, 'hours': '17'}
	date_time:  {'to': datetime.datetime(2019, 9, 7, 8, 0), 'from': datetime.datetime(2019, 9, 6, 15, 25)}
	city:  {'to': 'Прага', 'from': 'Москва'}


crawl_date_time:  2019-04-22 12:43:26.216543
price:  6221
	total_time:  {'minutes': '15', 'days': 0, 'hours': '23'}
	date_time:  {'to': datetime.datetime(2019, 9, 7, 8, 0), 'from': datetime.datetime(2019, 9

crawl_date_time:  2019-04-22 12:46:31.972295
price:  12403
	total_time:  {'minutes': '15', 'days': 0, 'hours': '8'}
	date_time:  {'to': datetime.datetime(2019, 9, 6, 14, 35), 'from': datetime.datetime(2019, 9, 6, 7, 20)}
	city:  {'to': 'Севилья', 'from': 'Москва'}


crawl_date_time:  2019-04-22 12:46:35.516548
price:  12403
	total_time:  {'minutes': 0, 'days': 0, 'hours': '18'}
	date_time:  {'to': datetime.datetime(2019, 9, 7, 0, 20), 'from': datetime.datetime(2019, 9, 6, 7, 20)}
	city:  {'to': 'Севилья', 'from': 'Москва'}


crawl_date_time:  2019-04-22 12:46:38.456627
price:  12518
	total_time:  {'minutes': '30', 'days': '1', 'hours': '16'}
	date_time:  {'to': datetime.datetime(2019, 9, 7, 20, 20), 'from': datetime.datetime(2019, 9, 6, 4, 50)}
	city:  {'to': 'Севилья', 'from': 'Москва'}


crawl_date_time:  2019-04-22 12:46:43.194960
price:  12518
	total_time:  {'minutes': '30', 'days': '1', 'hours': '16'}
	date_time:  {'to': datetime.datetime(2019, 9, 7, 20, 20), 'from': datetime.date

crawl_date_time:  2019-04-22 12:50:51.339499
price:  24711
	total_time:  {'minutes': '5', 'days': '1', 'hours': '6'}
	date_time:  {'to': datetime.datetime(2019, 9, 7, 21, 15), 'from': datetime.datetime(2019, 9, 6, 22, 10)}
	city:  {'to': 'Монреаль', 'from': 'Москва'}


crawl_date_time:  2019-04-22 12:50:53.912443
price:  24992
	total_time:  {'minutes': '30', 'days': '1', 'hours': '12'}
	date_time:  {'to': datetime.datetime(2019, 9, 7, 21, 15), 'from': datetime.datetime(2019, 9, 6, 15, 45)}
	city:  {'to': 'Монреаль', 'from': 'Москва'}


crawl_date_time:  2019-04-22 12:50:56.810013
price:  25232
	total_time:  {'minutes': '40', 'days': '1', 'hours': '17'}
	date_time:  {'to': datetime.datetime(2019, 9, 7, 21, 15), 'from': datetime.datetime(2019, 9, 6, 10, 35)}
	city:  {'to': 'Монреаль', 'from': 'Москва'}


crawl_date_time:  2019-04-22 12:50:59.181349
price:  25245
	total_time:  {'minutes': '58', 'days': '1', 'hours': '16'}
	date_time:  {'to': datetime.datetime(2019, 9, 7, 17, 18), 'from': 

...

	total_time:  {'minutes': '15', 'days': 0, 'hours': '18'}
	date_time:  {'to': datetime.datetime(2019, 9, 10, 13, 55), 'from': datetime.datetime(2019, 9, 9, 20, 40)}
	city:  {'to': 'Берлин', 'from': 'Москва'}


**********************************************************************************************************************
$$$$$   page loaded for 49 seconds   $$$$$
MOW → BRN 

**********************************************************************************************************************
$$$$$   page loaded for 39 seconds   $$$$$
MOW → BLQ 

crawl_date_time:  2019-04-22 15:45:33.420919
price:  7625
	total_time:  {'minutes': '35', 'days': 0, 'hours': '24'}
	date_time:  {'to': datetime.datetime(2019, 9, 10, 15, 50), 'from': datetime.datetime(2019, 9, 9, 16, 15)}
	city:  {'to': 'Болонья', 'from': 'Москва'}


crawl_date_time:  2019-04-22 15:45:36.778722
price:  9344
	total_time:  {'minutes': '20', 'days': 0, 'hours': '12'}
	date_time:  {'to': datetime.datetime(2019, 9, 9, 15, 0),

crawl_date_time:  2019-04-22 15:48:54.262825
price:  25967
	total_time:  {'minutes': '20', 'days': 0, 'hours': '20'}
	date_time:  {'to': datetime.datetime(2019, 9, 10, 10, 40), 'from': datetime.datetime(2019, 9, 9, 15, 20)}
	city:  {'to': 'Брно', 'from': 'Москва'}


crawl_date_time:  2019-04-22 15:48:58.295528
price:  27255
	total_time:  {'minutes': '25', 'days': '1', 'hours': '13'}
	date_time:  {'to': datetime.datetime(2019, 9, 10, 22, 35), 'from': datetime.datetime(2019, 9, 9, 10, 10)}
	city:  {'to': 'Брно', 'from': 'Москва'}


**********************************************************************************************************************
$$$$$   page loaded for 75 seconds   $$$$$
MOW → OST 

crawl_date_time:  2019-04-22 15:50:19.896048
price:  20370
	total_time:  {'minutes': '50', 'days': 0, 'hours': '9'}
	date_time:  {'to': datetime.datetime(2019, 9, 10, 0, 30), 'from': datetime.datetime(2019, 9, 9, 15, 40)}
	city:  {'to': 'Брюгге', 'from': 'Москва'}


crawl_date_time:  2019-

	total_time:  {'minutes': '30', 'days': 0, 'hours': '17'}
	date_time:  {'to': datetime.datetime(2019, 9, 10, 11, 15), 'from': datetime.datetime(2019, 9, 9, 18, 45)}
	city:  {'to': 'Валенсия', 'from': 'Москва'}


crawl_date_time:  2019-04-22 15:54:35.359239
price:  8825
	total_time:  {'minutes': '45', 'days': 0, 'hours': '23'}
	date_time:  {'to': datetime.datetime(2019, 9, 10, 17, 30), 'from': datetime.datetime(2019, 9, 9, 18, 45)}
	city:  {'to': 'Валенсия', 'from': 'Москва'}


crawl_date_time:  2019-04-22 15:54:38.085550
price:  9588
	total_time:  {'minutes': '55', 'days': '1', 'hours': '2'}
	date_time:  {'to': datetime.datetime(2019, 9, 10, 18, 55), 'from': datetime.datetime(2019, 9, 9, 17, 0)}
	city:  {'to': 'Валенсия', 'from': 'Москва'}


crawl_date_time:  2019-04-22 15:54:40.877603
price:  10173
	total_time:  {'minutes': '50', 'days': 0, 'hours': '21'}
	date_time:  {'to': datetime.datetime(2019, 9, 10, 0, 35), 'from': datetime.datetime(2019, 9, 9, 3, 45)}
	city:  {'to': 'Валенсия',

**********************************************************************************************************************
$$$$$   page loaded for 39 seconds   $$$$$
MOW → VRN 

crawl_date_time:  2019-04-22 15:58:03.371545
price:  8490
"Element not found (getDetailedFlightInfo):  Message: Unable to locate element: .ticket-segment__details
 "
	total_time:  {'minutes': '35', 'days': 0, 'hours': '3'}
	date_time:  {'to': datetime.datetime(2019, 9, 9, 14, 25), 'from': datetime.datetime(2019, 9, 9, 11, 50)}
	city:  {'to': 'Верона', 'from': 'Москва'}


crawl_date_time:  2019-04-22 15:58:05.993447
price:  7845
	total_time:  {'minutes': '25', 'days': 0, 'hours': '3'}
	date_time:  {'to': datetime.datetime(2019, 9, 9, 10, 0), 'from': datetime.datetime(2019, 9, 9, 7, 35)}
	city:  {'to': 'Верона', 'from': 'Москва'}


crawl_date_time:  2019-04-22 15:58:09.161102
price:  8490
	total_time:  {'minutes': '35', 'days': 0, 'hours': '3'}
	date_time:  {'to': datetime.datetime(2019, 9, 9, 14, 25), 'from': dateti

InvalidSessionIdException: Message: Tried to run command without establishing a connection


In [5]:
al_time:  {'minutes': '40', 'days': 0, 'hours': '6'}
	date_time:  {'to': datetime.datetime(2019, 9, 6, 10, 40), 'from': datetime.datetime(2019, 9, 6, 5, 0)}
	city:  {'to': 'Неаполь', 'from': 'Москва'}
#март
time_list_march = ['0103','0203','0303','0403','0503','0603','0703','0803','0903','1003',
                   '1103','1203','1303','1403','1503','1603','1703','1803','1903','2003',
                   '2103','2203','2303','2403','2503','2603','2703','2803','2903','3003','3103']

#апрель
time_list_april = ['0104','0204','0304','0404','0504','0604','0704','0804','0904','1004',
                   '1104','1204','1304','1404','1504','1604','1704','1804','1904','2004',
                   '2104','2204','2304','2404','2504','2604','2704','2804','2904','3004']

#май
time_list_may = ['0105','0205','0305','0405','0505','0605','0705','0805','0905','1005',
                 '1105','1205','1305','1405','1505','1605','1705','1805','1905','2005',
                 '2105','2205','2305','2405','2505','2605','2705','2805','2905','3005', '3105']

#июнь
time_list_june = ['0106','0206','0306','0406','0506','0606','0706','0806','0906','1006',
                  '1106','1206','1306','1406','1506','1606','1706','1806','1906','2006',
                  '2106','2206','2306','2406','2506','2606','2706','2806','2906','3006']

#июль
time_list_july = ['0107','0207','0307','0407','0507','0607','0707','0807','0907','1007',
                  '1107','1207','1307','1407','1507','1607','1707','1807','1907','2007',
                  '2107','2207','2307','2407','2507','2607','2707','2807','2907','3007', '3107']

#август
time_list_august = ['0108','0208','0308','0408','0508','0608','0708','0808','0908','1008',
                    '1108','1208','1308','1408','1508','1608','1708','1808','1908','2008',
                    '2108','2208','2308','2408','2508','2608','2708','2808','2908','3008', '3108']

#сентябрь
time_list_september = ['0109','0209','0309','0409','0509','0609','0709','0809','0909','1009',
                       '1109','1209','1309','1409','1509','1609','1709','1809','1909','2009',
                       '2109','2209','2309','2409','2509','2609','2709','2809','2909','3009']


city_list = {'BER':'Берлин', 
             'HAM':'Гамбург', 
             'BRE':'Бремен', 
             'HAJ':'Ганновер', 
             'LEJ':'Лейпциг', 
             'DUS':'Дюссельдорф', 
             'CGN':'Кёльн', 
             'NUE':'Нюрнберг', 
             'MUC':'Мюнхен', 
             'PRG':'Прага', 
             'BRQ':'Брно', 
             'KLV':'Карловы Вары', 
             'VIE':'Вена', 
             'SZG':'Зальцбург', 
             'INN':'Инсбрук', 
             'BUD':'Будапешт', 
             'BRU':'Брюссель', 
             'ANR':'Антверпен', 
             'OST':'Брюгге', 
             'NCE':'Ницца', 
             'LYS':'Лион', 
             'PAR':'Париж', 
             'MPL':'Монпелье', 
             'SXB':'Страсбург', 
             'GVA':'Женева', 
             'ZRH':'Цюрих', 
             'BRN':'Берн',
             'MIL':'Милан',
             'ROM':'Рим',
             'VCE':'Венеция',
             'VRN':'Верона',
             'NAP':'Неаполь',
             'TRN':'Турин',
             'FLR':'Флоренция',
             'PMO':'Палермо',
             'BLQ':'Болонья',
             'PEG':'Перуджа',
             'AGP':'Малага',
             'BCN':'Барселона',
             'PMI':'Майорка',
             'MAD':'Мадрид',
             'ALC':'Аликанте',
             'SVQ':'Севилья',
             'VLC':'Валенсия',
             'QCN':'Таррагона',
             'IBZ':'Ибица',
             'LON':'Лондон',
             'MAN':'Манчестер',
             'EDI':'Эдинбург',
             'GLA':'Глазго',
             'CBG':'Кембридж',
             'BFS':'Белфаст',
             'DUB':'Дублин',
             'ORC':'Корк',
             'AMS':'Амстердам',
             'RTM':'Роттердам',
             'EIN':'Эйндховен',
             'LCA':'Ларнака',
             'ECN':'Никосия',
             'PFO':'Пафос',
             'STO':'Стокгольм',
             'GOT':'Гётерберг'
             }

city_list = OrderedDict(sorted(city_list.items(), key=lambda kv: kv[1]))

In [417]:
def printBeautifulTicket(post):
    print('_id: ', post['_id'])
    print('crawl_date_time: ', post['crawl_date_time'])
    print('flights: ')
    for flight in post['flights']:
        print('\tcity: ')
        print('\t\tfrom\t: ', flight['city']['from'])
        print('\t\tto\t: ', flight['city']['to'])
        print('\tdate_time: ')
        print('\t\tfrom\t: ', flight['date_time']['from'])
        print('\t\tto\t: ', flight['date_time']['to'])
        print('\tdetailes: ')
        for period in flight['details']:
            if period['type'] == 'leg':
                print('\t\tairline\t: ', period['airline'])
                print('\t\tairport: ')
                print('\t\t\tfrom: ')
                print('\t\t\t\tiata\t: ', period['airport']['from']['iata'])
                print('\t\t\t\tname\t: ', period['airport']['from']['name'])
                print('\t\t\tto: ')
                print('\t\t\t\tiata\t: ', period['airport']['to']['iata'])
                print('\t\t\t\tname\t: ', period['airport']['to']['name'])
                print('\t\tdate_time: ')
                print('\t\t\tfrom\t: ', period['date_time']['from'])
                print('\t\t\tto\t: ', period['date_time']['to'])
                print('\t\tflight_number\t: ', period['flight_number'])
                print('\t\ttype\t: ', period['type'])
            if period['type'] == 'stop':
                print('\t\tplace\t: ', period['place'])
                print('\t\ttime: ')
                print('\t\t\thours\t: ', period['time']['hours'])
                print('\t\t\tminutes\t: '), period['time']['minutes']
                
    print('\n\n')

In [1]:
for post in collection.find({'price':61765}):
    printBeautifulTicket(post)

NameError: name 'collection' is not defined

In [ ]:
for post in collection.find({'price':61765}):
    pprint(post)